# CDC - Change Data Capture

### Import data into 

In [1]:
fd = open('project/Dump.txt', 'r')
sqlFile = fd.read()
fd.close()

sqlCommands = sqlFile.split(';')

In [58]:
import mysql.connector

db_connection = mysql.connector.connect(
    host='10.10.0.100',
    user="root",
    password="1",
    database="")
db_cursor = db_connection.cursor()

In [59]:
del sqlCommands[-1]
for command in sqlCommands:
    try:
        if command != '':
            db_cursor.execute(command)
    except mysql.connector.Error as err:
        print("Something went wrong: {}".format(err))
db_connection.commit()   
db_connection.close()

### Glue Job

In [1]:
import sys
from pyspark.sql.functions import when
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('CDC').getOrCreate()

23/05/31 07:11:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/31 07:11:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [21]:
fldf = spark.read.csv('project/LOAD00000001.csv')
fldf = fldf.withColumnRenamed('_c0','id')\
.withColumnRenamed('_c1','FullName')\
.withColumnRenamed('_c2','City')

In [22]:
fldf.write.mode('overwrite').options(header='True').csv('project/output/finalFile.csv')

In [23]:
udf = spark.read.csv('project/20230531-041611011.csv')
udf = udf.withColumnRenamed('_c0','action')\
.withColumnRenamed('_c1','id')\
.withColumnRenamed('_c2','FullName')\
.withColumnRenamed('_c3','City')

In [24]:
ffdf = spark.read.options(header='True').csv('project/output/finalFile.csv')
ffdf.count()

101

In [25]:
columns = ['id', 'FullName', 'City']
for row in udf.collect():
    if row['action'] == 'U':
        ffdf = ffdf.withColumn("FullName", when(ffdf["id"] == row["id"], row["FullName"])\
                               .otherwise(ffdf ["FullName"]))
        ffdf = ffdf.withColumn("City", when(ffdf["id"] == row["id"], row["City"])\
                               .otherwise(ffdf["City"]))
        
    if row['action'] == 'I':
        insertData = [list(row[1:])]
        newdf = spark.createDataFrame(insertData, columns)
        ffdf = ffdf.union(newdf)
        
    if row['action'] == 'D':
        ffdf = ffdf.filter(ffdf.id != row['id'])

In [26]:
ffdf.count()

103

In [27]:
ffdf.cache().show(1)

+---+----------------+-------------+
| id|        FullName|         City|
+---+----------------+-------------+
|  0|Herman Zimmerman|Oklahoma City|
+---+----------------+-------------+
only showing top 1 row



In [28]:
ffdf.write.mode('overwrite').options(header='True').csv('project/output/finalFile.csv')

In [29]:
ffdf1 = spark.read.options(header='True').csv('project/output/finalFile.csv')
ffdf1.count()

103